### Multiple Days

In [4]:
start_date = '2023-9-1' 
end_date = '2023-09-24'

In [5]:
from datetime import datetime
import pandas as pd
latest_data_available = start_date
from_date  = datetime.strptime(latest_data_available, '%Y-%m-%d')
to = end_date
to_date = datetime.strptime(to, '%Y-%m-%d')

In [6]:
date_to_start_scraping_from  = from_date.date() + pd.to_timedelta(1, unit="D")
dates = pd.date_range(start = date_to_start_scraping_from, end = to)
dates = [d for d in dates if not d.isoweekday() in [5,6]] # List excluding Friday and Saturday
print(len(dates))

16


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.sharesansar.com/top-brokers')
# driver.implicitly_wait(15)
 # Starting Nepalipaisa websitee



blank_df = []
for date in dates:
    driver.find_element(By.XPATH, '//*[@id="date"]').clear()
    driver.find_element(By.XPATH, '//*[@id="date"]').send_keys(f'{date.date()}')# 
    driver.find_element(By.XPATH, '//*[@id="btn_topbrokers_submit"]').click()
    #     time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    todays_price = soup.find('table', id = 'myTable')
    time.sleep(1)
    if len(todays_price.findAll('tr'))<10:
        pass
    else:
        output_rows = []
        for table_row in todays_price.findAll('tr'): # looping through all the table rows in a page
            columns = table_row.findAll('td') #finding the cell values of every row in the table
            output_row = []
            for column in columns: # looping through each cellvalue data in a row 
                output_row.append(column.text)
            output_rows.append(output_row)
            headers_list = []
            for headers in todays_price.find_all('th'):
                headers_list.append(headers.text)
        todays_price_dataframe = pd.DataFrame(output_rows)[1:]
        todays_price_dataframe.columns = headers_list
    #     todays_price_dataframe['Symbol'] = todays_price_dataframe['Symbol'].str.replace("\n", "")
        todays_price_dataframe.set_index('S.N.', inplace = True)
        todays_price_dataframe['Date'] = date
        blank_df.append(todays_price_dataframe)
#         time.sleep(1)
        print(date)

driver.quit()


In [ ]:
# pd.concat(blank_df).to_csv('turnover19-20.csv')